In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

# SQL (SQLAlchemy)

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [11]:
# 사용자 미시지를 추가
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 종진이야. 나는 개발자야. 앞으로 잘 부탁해!"
)

# AI 메시지를 추가
chat_message_history.add_ai_message(
    "안녕 종진, 만나서 반가와. 나도 잘 부탁해"
)

In [12]:
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내 이름은 종진이야. 나는 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕? 만나서 반가워. 내 이름은 종진이야. 나는 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 종진, 만나서 반가와. 나도 잘 부탁해', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕? 만나서 반가워. 내 이름은 종진이야. 나는 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 종진, 만나서 반가와. 나도 잘 부탁해', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕? 만나서 반가워. 내 이름은 종진이야. 나는 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 종진, 만나서 반가와. 나도 잘 부탁해', additional_kwargs={}, response_metadata={})]

# Chain 에 적용

In [13]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Out are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)

In [16]:
chain = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0) | StrOutputParser()

In [17]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

In [18]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True
    )
]

In [21]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history, # 대화 기록을 가져오는 함수를 설정
    input_messages_key="question", # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history", # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields, # 대화 기록 조회시 참고할 파라미터를 설정
)

In [22]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [24]:
# 질문과 config를 전달하여 실행
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 종진이야"}, config)

'안녕하세요, 종진님! 반갑습니다. 어떻게 도와드릴까요?'

In [25]:
# 후속 질문을 실해합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'종진님이라고 하셨습니다! 맞나요?'

In [26]:
# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config를 전달하여 실행
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!'